<a href="https://colab.research.google.com/github/spannozzo/posting_article_ds/blob/main/posting_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.113.5.90:8470


REPLICAS:  8


In [4]:
!git clone https://github.com/spannozzo/posting_article_ds.git

Cloning into 'posting_article_ds'...
remote: Enumerating objects: 17, done.
remote: Total 17 (delta 0), reused 0 (delta 0), pack-reused 17
Unpacking objects: 100% (17/17), done.


In [6]:
!unrar e /content/posting_article_ds/data/calendar.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/posting_article_ds/data/calendar.rar

Extracting  calendar.csv                                                  22% 45% 68% 90% 99%  OK 
All OK


In [8]:
!unrar e /content/posting_article_ds/data/listings.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/posting_article_ds/data/listings.rar

Extracting  listings.csv                                                  45% 90% 99%  OK 
All OK


In [11]:
!unrar x /content/posting_article_ds/data/reviews.part1.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/posting_article_ds/data/reviews.part1.rar

Extracting  reviews.csv                                                    3%  7% 10% 14% 18% 21% 25% 29% 32% 36% 40% 43% 45%

Extracting from /content/posting_article_ds/data/reviews.part2.rar

...         reviews.csv                                                   49% 52% 56% 60% 63% 67% 71% 74% 78% 81% 85% 89% 91%

Extracting from /content/posting_article_ds/data/reviews.part3.rar

...         reviews.csv                                                   94% 98% 99%  OK 
All OK


In [12]:
import pandas as pd

df_cal = pd.read_csv('/content/calendar.csv')
df_cal.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,49955080,2022-06-07,f,$66.00,$66.00,1.0,365.0
1,49955080,2022-06-08,f,$66.00,$66.00,1.0,365.0
2,49955080,2022-06-09,f,$66.00,$66.00,1.0,365.0
3,49955080,2022-06-10,t,$66.00,$66.00,1.0,365.0
4,49955080,2022-06-11,t,$66.00,$66.00,1.0,365.0


In [13]:
df_cal.columns

Index(['listing_id', 'date', 'available', 'price', 'adjusted_price',
       'minimum_nights', 'maximum_nights'],
      dtype='object')

In [15]:
df_cal.isna().sum()

listing_id        0
date              0
available         0
price             0
adjusted_price    0
minimum_nights    7
maximum_nights    7
dtype: int64

In [16]:
df_cal['available'].value_counts()

t    4425791
f    4301362
Name: available, dtype: int64

In [19]:
df_cal[df_cal['available']=='t'].head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
3,49955080,2022-06-10,t,$66.00,$66.00,1.0,365.0
4,49955080,2022-06-11,t,$66.00,$66.00,1.0,365.0
5,49955080,2022-06-12,t,$66.00,$66.00,1.0,365.0
11,49955080,2022-06-18,t,$66.00,$66.00,1.0,365.0
12,49955080,2022-06-19,t,$66.00,$66.00,1.0,365.0


In [20]:
df_cal[df_cal['available']=='f'].head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,49955080,2022-06-07,f,$66.00,$66.00,1.0,365.0
1,49955080,2022-06-08,f,$66.00,$66.00,1.0,365.0
2,49955080,2022-06-09,f,$66.00,$66.00,1.0,365.0
6,49955080,2022-06-13,f,$66.00,$66.00,1.0,365.0
7,49955080,2022-06-14,f,$66.00,$66.00,1.0,365.0
